In [ ]:
from pprint import pprint
from datasets import load_dataset
squad = load_dataset("squad")
for item in squad["train"][1].items():
    print(item[0])
    pprint(item[1])
    print("="*20)

Dataset squad downloaded and prepared to /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453. Subsequent calls will reuse this data.



context
('Architecturally, the school has a Catholic character. Atop the Main '
 "Building's gold dome is a golden statue of the Virgin Mary. Immediately in "
 'front of the Main Building and facing it, is a copper statue of Christ with '
 'arms upraised with the legend "Venite Ad Me Omnes". Next to the Main '
 'Building is the Basilica of the Sacred Heart. Immediately behind the '
 'basilica is the Grotto, a Marian place of prayer and reflection. It is a '
 'replica of the grotto at Lourdes, France where the Virgin Mary reputedly '
 'appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive '
 '(and in a direct line that connects through 3 statues and the Gold Dome), is '
 'a simple, modern stone statue of Mary.')
question
'What is in front of the Notre Dame Main Building?'
id
'5733be284776f4190066117f'
title
'University_of_Notre_Dame'
answers
{'answer_start': [188], 'text': ['a copper statue of Christ']}


In [ ]:
squad = load_dataset("squad_v2")
squad

Dataset squad_v2 downloaded and prepared to /root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d. Subsequent calls will reuse this data.


DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 130319
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11873
    })
})

In [ ]:
from transformers import AutoTokenizer

model = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model)

In [ ]:
max_length = 384
doc_stride = 128
example = squad["train"][173]
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    stride=doc_stride
)

In [ ]:
len(tokenized_example['input_ids'])

2

In [ ]:
for input_ids in tokenized_example["input_ids"][:2]:
    print(tokenizer.decode(input_ids))
    print("-"*50)

[CLS] beyonce got married in 2008 to whom? [SEP] on april 4, 2008, beyonce married jay z. she publicly revealed their marriage in a video montage at the listening party for her third studio album, i am... sasha fierce, in manhattan's sony club on october 22, 2008. i am... sasha fierce was released on november 18, 2008 in the united states. the album formally introduces beyonce's alter ego sasha fierce, conceived during the making of her 2003 single " crazy in love ", selling 482, 000 copies in its first week, debuting atop the billboard 200, and giving beyonce her third consecutive number - one album in the us. the album featured the number - one song " single ladies ( put a ring on it ) " and the top - five songs " if i were a boy " and " halo ". achieving the accomplishment of becoming her longest - running hot 100 single in her career, " halo "'s success in the us helped beyonce attain more top - ten singles on the list than any other woman during the 2000s. it also included the suc

In [ ]:
def prepare_train_features(examples, pad_on_right=True): 
    tokenized_examples = tokenizer( 
        examples["question" if pad_on_right else "context"], 
        examples["context" if pad_on_right else "question"], 
        truncation="only_second" if pad_on_right else "only_first", 
        max_length=max_length, 
        stride=doc_stride, 
        return_overflowing_tokens=True, 
        return_offsets_mapping=True, 
        padding="max_length", 
    ) 
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping") 
    offset_mapping = tokenized_examples.pop("offset_mapping") 
    tokenized_examples["start_positions"] = [] 
    tokenized_examples["end_positions"] = [] 
    for i, offsets in enumerate(offset_mapping): 
        input_ids = tokenized_examples["input_ids"][i] 
        cls_index = input_ids.index(tokenizer.cls_token_id) 
        sequence_ids = tokenized_examples.sequence_ids(i) 
        sample_index = sample_mapping[i] 
        answers = examples["answers"][sample_index] 
        if len(answers["answer_start"]) == 0: 
            tokenized_examples["start_positions"].append(cls_index) 
            tokenized_examples["end_positions"].append(cls_index) 
        else: 
            start_char = answers["answer_start"][0] 
            end_char = start_char + len(answers["text"][0]) 
            token_start_index = 0 
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0): 
                token_start_index += 1 
            token_end_index = len(input_ids) - 1 
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0): 
                token_end_index -= 1 
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char): 
                tokenized_examples["start_positions"].append(cls_index) 
                tokenized_examples["end_positions"].append(cls_index) 
            else: 
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char: 
                    token_start_index += 1 
                tokenized_examples["start_positions"].append(token_start_index - 1) 
                while offsets[token_end_index][1] >= end_char: 
                    token_end_index -= 1 
                tokenized_examples["end_positions"].append(token_end_index + 1) 
    return tokenized_examples

In [ ]:
tokenized_datasets = squad.map(
    prepare_train_features, batched=True, remove_columns=squad["train"].column_names)

In [ ]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer 
model = AutoModelForQuestionAnswering.from_pretrained("distilbert-base-uncased")

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
args = TrainingArguments(
    "test-squad",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01
)

In [ ]:
from transformers import default_data_collator
data_collator = default_data_collator 

max_size = 10000
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer
)

In [ ]:
trainer.train()

***** Running training *****


  Num examples = 131754


  Num Epochs = 3


  Instantaneous batch size per device = 16


  Total train batch size (w. parallel, distributed & accumulation) = 16


...

Configuration saved in test-squad/checkpoint-500/config.json


Model weights saved in test-squad/checkpoint-500/pytorch_model.bin


tokenizer config file saved in test-squad/checkpoint-500/tokenizer_config.json


Special tokens file saved in test-squad/checkpoint-500/special_tokens_map.json


KeyboardInterrupt: 

In [ ]:
trainer.save_model("distillBERT_SQUAD")

Saving model checkpoint to distillBERT_SQUAD


Configuration saved in distillBERT_SQUAD/config.json


Model weights saved in distillBERT_SQUAD/pytorch_model.bin


tokenizer config file saved in distillBERT_SQUAD/tokenizer_config.json


Special tokens file saved in distillBERT_SQUAD/special_tokens_map.json


In [ ]:
from transformers import pipeline 
qa_model = pipeline('question-answering', 
    model='distilbert-base-cased-distilled-squad', 
    tokenizer='distilbert-base-cased')

https://huggingface.co/distilbert-base-cased-distilled-squad/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpewyfx7rg


storing https://huggingface.co/distilbert-base-cased-distilled-squad/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/81e8dfe090123eff18dd06533ead3ae407b82e30834c50c7c82c2305ce3ace12.ca0305b1f128274fa0c6e4859d1c1477d0e34a20be25da95eea888b30ece9cf3


creating metadata file for /root/.cache/huggingface/transformers/81e8dfe090123eff18dd06533ead3ae407b82e30834c50c7c82c2305ce3ace12.ca0305b1f128274fa0c6e4859d1c1477d0e34a20be25da95eea888b30ece9cf3


loading configuration file https://huggingface.co/distilbert-base-cased-distilled-squad/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/81e8dfe090123eff18dd06533ead3ae407b82e30834c50c7c82c2305ce3ace12.ca0305b1f128274fa0c6e4859d1c1477d0e34a20be25da95eea888b30ece9cf3


Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForQuestionAnswering"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": true,
  "tie_weights_": true,
  "transformers_version": "4.11.3",
  "vocab_size": 28996
}



...

loading file https://huggingface.co/distilbert-base-cased/resolve/main/added_tokens.json from cache at None


loading file https://huggingface.co/distilbert-base-cased/resolve/main/special_tokens_map.json from cache at None


loading file https://huggingface.co/distilbert-base-cased/resolve/main/tokenizer_config.json from cache at /root/.cache/huggingface/transformers/81e970e5e6ec68be12da0f8f3b2f2469c78d579282299a2ea65b4b7441719107.ec5c189f89475aac7d8cbd243960a0655cfadc3d0474da8ff2ed0bf1699c2a5f


loading configuration file https://huggingface.co/distilbert-base-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/ebe1ea24d11aa664488b8de5b21e33989008ca78f207d4e30ec6350b693f073f.302bfd1b5e031cc1b17796e0b6e5b242ba2045d31d00f97589e12b458ebff27a


Model config DistilBertConfig {
  "activation": "gelu",
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.11.3",
  "vocab_size": 28996
}



In [ ]:
question = squad["validation"][0]["question"] 
context = squad["validation"][0]["context"] 
print("Question:") 
print(question) 
print("Context:") 
print(context)

Question:
In what country is Normandy located?
Context:
The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.


In [ ]:
qa_model(question=question, context=context)


{'score': 0.9889379143714905, 'start': 159, 'end': 165, 'answer': 'France'}